## To-do

+ 对象：
    + 五绝（二联，四句）
    + 五律（四联，八句）
+ 原始诗歌文本预处理（分字）
+ 采用《平水韵》韵书判断平仄（问题：多音字）
+ 判断符合格律：
    + 五绝（律）平起首句入韵
    + 五绝（律）仄起首句入韵
    + 五绝（律）平起首句不入韵
    + 五绝（律）仄起首句不入韵
+ 判断押韵
    + 失粘、失对、三平、孤平
+ 编码使符合 TEI Verse 格式
    + 解读 TEI Verse 标签******

In [1]:
# encoding=utf-8
import jieba
import nltk
import json
import re

In [2]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [3]:
with open("cope/data/rhymebooks.json") as output:
    rhymes = json.load(output)
with open("cope/data/TC2SC.json") as output:
    T2C = json.load(output)
with open("cope/data/kangxi.json") as output:
    Dict = json.load(output)

In [4]:
print(Dict['煜'])

【唐韻】【集韻】【韻會】【正韻】□余六切，音育。【說文】燿也。【前漢班固敘傳】其餘焱飛景附，煜霅其閒者，蓋不可勝載。【註】師古曰：煜霅，光貌。　又【玉篇】火焰也。【廣韻】火光。　又【玉篇】盛貌。【班固東都賦】管絃曄煜。【註】曄煜，盛貌。　又【廣韻】爲立切【集韻】域及切【韻會】弋入切，□音熠。【廣韻】火貌。　【廣韻】同焴。


In [5]:
# 《平水韵》有入声，《中华新韵》无。
# 诗用《平水韵》，词用《词林正韵》
P_PING_unflattened = rhymes['平水韵'][0]
Z_PING_unflattened = rhymes['平水韵'][1]
P_PING = flatten(rhymes['平水韵'][0])
Z_PING = flatten(rhymes['平水韵'][1])
# P_ZHXY = flatten(rhymes['中华新韵'][0])
# Z_ZHXY = flatten(rhymes['中华新韵'][1])

In [22]:
raw1 = "白日依山尽，黄河入海流。欲穷千里目，更上一层楼。"
raw2 = "迟日江山丽，春风花草香。 泥融飞燕子， 沙暖睡鸳鸯。"
raw3 = "鸣筝金粟柱, 素手玉房前, 欲得周郎顾, 时时误拂弦"
raw4 = "花明绮陌春，柳拂御沟新。为报辽阳客，流芳不待人。"
raw5 = "北斗七星高，哥舒夜带刀。至今窥牧马，不敢过临洮。"
raw6 = "离离原上草，一岁一枯荣。野火烧不尽，春风吹又生。"

In [23]:
raw7 = "泠泠七弦上，静听松风寒。古调虽自爱，今人多不弹。"

In [9]:
def preprocess(raw):
    clean = re.sub(r'[，。, .]', "", raw)
    tokens = [token for token in clean if token]
    tones  = []
    for t in tokens:
        if t in P_PING:
            if t in Z_PING:
                tones.append('1/0') 
            else:
                tones.append('1') 
        else:
            tones.append('0')
    return tokens,tones

In [67]:
metrics = []

# 五绝平起首句入韵
metrics.append([
    '1','1','1/0','0','1',
    '1/0','0','0','1','1',
    '1/0','0','1','1','0',
    '1','1','1/0','0','1'])
# 五绝平起首句不入韵
metrics.append([
    '1/0','1','1','0','0',
    '1/0','0','0','1','1',
    '1/0','0','1','1','0',
    '1','1','1/0','0','1'])
# 五绝仄起首句入韵
metrics.append([
    '1/0','0','0','1','1',
    '1','1','1/0','0','1',
    '1/0','1','1','0','0',
    '1/0','0','0','1','1'])
# 五绝仄起首句不入韵
metrics.append([
    '1/0','0','1','1','0',
    '1','1','1/0','0','1',
    '1/0','1','1','0','0',
    '1/0','0','0','1','1'])

In [90]:
def metricsAnalytics(verse, metrics):
    tokens, tones = verse
    print('五绝') 
    count = 0 
    RBOOK = P_PING_unflattened
    if tones[1] == '1':
        print('平起') 
        metric = metrics[:2]
    else:
        print('仄起')
        metric = metrics[2:]
        
    for k,v in enumerate(RBOOK):
        if tokens[9] in v and tokens[19] in v:
            if tones[4] == '1':
                num_metrics = 0
                print('首句入韵')
            else:
                num_metrics = 1
                print('首句不入韵')

                for k,(l1,l2) in enumerate(zip(tones,metric[num_metrics])):
                    if l1 == '1/0':
                        print("[检测到多音字",tokens[k],"]")
                    elif l2 == '1/0':
                        pass
                    elif l1 != l2 and k%5 != 0:
                        if l1 == '0':
                            print(tokens[k],"失配,建议：仄")
                        else:
                            print(tokens[k],"失配,建议：平")
                        count += 1
   
    print('失配词数',count)

In [91]:
verse1 = preprocess(raw1)
verse2 = preprocess(raw2)
verse3 = preprocess(raw3)
verse4 = preprocess(raw4)
verse5 = preprocess(raw5) 
verse6 = preprocess(raw6)
verse7 = preprocess(raw7)

In [92]:
metricsAnalytics(verse1, metrics)

五绝
仄起
首句不入韵
[检测到多音字 更 ]
失配词数 0


In [93]:
metricsAnalytics(verse2, metrics)

五绝
仄起
首句不入韵
[检测到多音字 丽 ]
[检测到多音字 泥 ]
[检测到多音字 燕 ]
失配词数 0


In [94]:
metricsAnalytics(verse3, metrics)

五绝
平起
首句不入韵
失配词数 0


In [95]:
metricsAnalytics(verse4, metrics)

五绝
平起
首句入韵
失配词数 0


In [96]:
metricsAnalytics(verse5, metrics)

五绝
仄起
首句入韵
失配词数 0


In [97]:
metricsAnalytics(verse6, metrics)

五绝
平起
首句不入韵
[检测到多音字 不 ]
[检测到多音字 吹 ]
失配词数 0


In [98]:
metricsAnalytics(verse7, metrics)

五绝
平起
首句不入韵
七 失配,建议：仄
弦 失配,建议：平
[检测到多音字 听 ]
松 失配,建议：平
[检测到多音字 调 ]
自 失配,建议：仄
[检测到多音字 不 ]
[检测到多音字 弹 ]
失配词数 4
